In [16]:
import os
from transformers import BertTokenizer, BertForSequenceClassification

model_dir = "/root/autodl-tmp/bert_cls_final_en_uncased"
base_model_dir = "/root/autodl-tmp/bert_cls_final_en_uncased"   # 你左侧文件树显示的路径

print("cwd:", os.getcwd())
print("base exists:", os.path.exists(base_model_dir))
print("files:", os.listdir(base_model_dir)[:10])

tokenizer = BertTokenizer.from_pretrained(base_model_dir, local_files_only=True)
model = BertForSequenceClassification.from_pretrained(model_dir, local_files_only=True)
model.eval()
print("loaded ok")


cwd: /root/autodl-tmp/bert_cls_final
base exists: True
files: ['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'training_args.bin']
loaded ok


In [17]:
import torch
def get_label_from_id(pred_id):
    id2label = model.config.id2label
    # 可能是 dict[int->str] 或 dict[str->str]，也可能是 list
    if isinstance(id2label, dict):
        return id2label.get(pred_id, id2label.get(str(pred_id), str(pred_id)))
    elif isinstance(id2label, list):
        return id2label[pred_id]
    else:
        return str(pred_id)


import torch

def predict(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.softmax(logits, dim=-1)[0].tolist()
    pred_id = int(torch.argmax(logits, dim=-1).item())
    pred_label = get_label_from_id(pred_id)

    return pred_label, probs



In [18]:

text = "The company’s troubles are now firmly behind it."
label, probs = predict(text)

print("文本:", text)
print("预测类别:", label)
print("概率:", probs)


文本: The company’s troubles are now firmly behind it.
预测类别: neutral
概率: [0.008705556392669678, 0.9709624648094177, 0.02033201791346073]


In [19]:
inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
model(**inputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8431,  2.8713, -0.9948]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)